# Plot Embeddings

In [23]:
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib qt
import matplotlib.pyplot as plt
import plotly.express as px
import itertools

In [24]:
## globals args
data_root_dir = os.getcwd() + '/plots_vae/'
vae_model = 'optimal-laggy-80-filled-s20'

agent_models = [
               'optimal',
               'laggy'
               ]

In [33]:
## load embeddings
embeddings = []
embeddings_sep = []
labels = []
label_counts = []

for idx, agent in enumerate(agent_models):
    embedding_dir = data_root_dir + vae_model + '/' + agent + '/'
    embed = np.load(embedding_dir + 'embeddings-test.npy')
    labels.append([idx] * len(embed))
    label_counts.append(len(embed))
    embeddings.append(embed)
    # embeddings_sep.append((embed - embed.mean(0, keepdims=True)) / embed.std(axis=0))
    embeddings_sep.append(embed / np.linalg.norm(embed, 2))

embeddings = np.concatenate(embeddings)
embeddings = (embeddings - embeddings.mean(0, keepdims=True)) / embeddings.std(axis=0)
labels = list(itertools.chain(*labels))
agent_dict = {0: "optimal", 1:"laggy"} 
models = list(map(agent_dict.get, labels))

In [34]:
# pca = PCA(n_components=3)
pca = PCA(n_components=2)
embeds_pca = pca.fit_transform(embeddings)

## draw PCA using matplotlib
# ax= plt.figure().add_subplot(projection='3d')
# # ax= plt.figure().add_subplot()

# ax.scatter(reduced_embeds[:,0],
#             reduced_embeds[:,1],
#             reduced_embeds[:,2],
#             c = colors)

# plt.show()

In [36]:
## draw PCA using plotly

df_pca = pd.DataFrame(embeds_pca, columns=['pca_1', 'pca_2'])
# df_pca = pd.DataFrame(embeds_pca, columns=['pca_1', 'pca_2', 'pca_3'])
df_pca['agent_model'] = models


# fig = px.scatter_matrix(df_pca,
#                         labels=labels,
#                         dimensions=['pca_1', 'pca_2', 'pca_3'],
#                         color='agent_model')
# fig.update_traces(diagonal_visible=False)

# fig = px.scatter_3d(df_pca,
#                     x='pca_1', y='pca_2', z='pca_3',
#                     color='agent_model')

fig = px.scatter(df_pca, 
                 x="pca_1", y="pca_2",
                #  color_discrete_sequence=["#14213D", "#FCA311", "#976391"],
                 color="agent_model",
                 title='PCA of embeddings -- test data',)
fig.update_layout(margin=dict(l=25, r=25, t=25, b=25),
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  title_x=0.5)

fig.show()

In [8]:
## similarity matrix

sim_matrix = np.zeros([len(agent_models), len(agent_models)])
for l1 in range(len(agent_models)):
    for l2 in range(len(agent_models)):
        if l1 != l2:
            sim_matrix[l1, l2] = cosine_similarity(embeddings_sep[l1], embeddings_sep[l2]).mean()
        else:
            mask = np.ones([len(embeddings_sep[l1]),len(embeddings_sep[l2])], dtype=bool)
            np.fill_diagonal(mask, 0)
            sim_matrix[l1, l2] = (cosine_similarity(embeddings_sep[l1], embeddings_sep[l2])*mask).mean()

## plot similarity matrix
fig = px.imshow(sim_matrix,
                labels=dict(x="agent type", y="agent type", color="similarity"),
                x=['optimal', 'laggy'],
                y=['optimal', 'laggy'],
                text_auto=True
               )
fig.update_xaxes(side="top")
fig.show()

In [21]:
color_dict = {0:'#14213D', 1:'#FCA311', 2:'#976391'}
# 0: 14213D
# 1: FCA311
# 2: 976391
colors = list(map(color_dict.get, labels))

# tsne = TSNE(3, verbose=1)
tsne = TSNE(2, verbose=1)
tsne_proj = tsne.fit_transform(embeddings)

# ax = plt.figure().add_subplot(projection='3d')
ax = plt.figure().add_subplot()
plt.scatter(tsne_proj[:,0],
            tsne_proj[:,1],
            # tsne_proj[:,2],
            color=colors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 192 samples in 0.000s...
[t-SNE] Computed neighbors for 192 samples in 0.011s...
[t-SNE] Computed conditional probabilities for sample 192 / 192
[t-SNE] Mean sigma: 6.709093
[t-SNE] KL divergence after 250 iterations with early exaggeration: 52.422657
[t-SNE] KL divergence after 1000 iterations: 0.825109


# Plot Trajectory

In [9]:
import numpy as np
import pandas as pd
import os
# %matplotlib qt
import matplotlib.pyplot as plt
# import plotly.express as px
# import itertools

import skvideo.io

In [38]:
data_root_dir = os.getcwd() + '/data_videos/'
agent = 'laggy'

root_dir = data_root_dir + agent + '-test/'
file = f'final-model-ppo-LunarLander-v2-{agent}-0.mp4'
frames = skvideo.io.vread(root_dir + file)
truth_data = np.load(root_dir+'env_grids.npy')[0]
agent_poss = np.stack(np.load(root_dir+'agent_poss.npz')['arr_0'])


print('done')

done


In [37]:
agent_poss['arr_0'].shape

(142, 2)